### Auxiliando os Times de Marketing e Pricing da Empresa de Delivery

A gente trabalha em uma empresa de delivery, num time de dados centralizado que atende a diversas áreas e recebemos algumas demandas que serão respondidas por meio da Análise dos dados.

In [4]:
import pandas as pd

1. Numa ação de marketing, para atrair mais entregadores, vamos dar uma bonificação para os 20 entregadores que possuem maior distância percorrida ao todo. A bonificação vai variar de acordo com o tipo de profissional que ele é e o modelo que ele usa para se locomover (moto, bike, etc). Levante essas informações.

In [68]:
drivers = pd.read_csv('./data/drivers.csv')
deliveries = pd.read_csv('./data/deliveries.csv')

In [ ]:
# Filtrando somente as entregas realizadas
deliveries = deliveries[deliveries['delivery_status'] == 'DELIVERED']

# Excluindo linhas com ids não informados
deliveries = deliveries.dropna(subset=["driver_id"])

# Modificando os ids para ser do tipo int
deliveries["driver_id"] = deliveries["driver_id"].astype(int)


In [77]:
# Filtrando os 20 entregadores com maior distância percorrida.
top20 = deliveries.groupby("driver_id")["delivery_distance_meters"].sum().sort_values(ascending=False).head(20)